In [95]:
from ortools.linear_solver import pywraplp
import pandas as pd

In [96]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [97]:
x1 = solver.NumVar(0,100000, "x1")
x2 = solver.NumVar(0,100000, "x2")
x3 = solver.NumVar(0,100000, "x3")

In [98]:
solver.Add(x1 + x2 + x3 <=3   , "cons1")
solver.Add(x1 + 4*x2 + 7*x3 <= 9, "cons2")

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x00000277E8776FD0> >

In [99]:
solver.Maximize(2*x1 + 3*x2 + x3)

In [100]:
solver.Solve()

0

In [101]:
if solver.Solve() == pywraplp.Solver.OPTIMAL:
    print("x1 = ", x1.solution_value(), "\nx2 = ", x2.solution_value(), "\nx3 = ", x3.solution_value(), 
          "\n",solver.Objective().Value())

x1 =  0.999999999999999 
x2 =  2.0000000000000004 
x3 =  0.0 
 8.0


In [102]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack': c.ub() - activities[i]} for i, c in enumerate(solver.constraints())]
print(pd.DataFrame(o))

    Name  shadow price         slack
0  cons1      1.666667  4.440892e-16
1  cons2      0.333333  0.000000e+00


# Problem solved now let's begin sensitivity analysis

In [103]:
# from which point product 3 production is profitable

x3_coff_lower = 2
x3_coff_upper = 10

for i in range(x3_coff_lower, x3_coff_upper):
    
    solver.Maximize(2*x1 + 3*x2 + i*x3)
    solver.Solve()
    
    if solver.Solve() == pywraplp.Solver.OPTIMAL:
        print(x1.solution_value(), x2.solution_value(), x3.solution_value())
        if round(x3.solution_value()) > 0:
            print('From price ', i, ' x3 is profitable')
            break

0.999999999999999 2.0000000000000004 0.0
0.999999999999999 2.0000000000000004 0.0
0.999999999999999 2.0000000000000004 0.0
1.9999999999999996 0.0 1.0000000000000002
From price  5  x3 is profitable


Unfortunately, OR-Tools' MPSolver interface doesn't directly support sensitivity analysis in its API. However, if you're using a solver backend that supports it (like Gurobi), you can leverage the backend's specific methods to get this information. If you specifically need sensitivity ranges within OR-Tools, you'd have to implement the analysis manually by perturbing the coefficients and resolving the model repeatedly.

In [115]:
import gurobipy as gp

# Define your model with Gurobi directly
model = gp.Model()

# Add variables
x1 = model.addVar(lb=0, name="x1")
x2 = model.addVar(lb=0, name="x2")
x3 = model.addVar(lb=0, name="x3")

# Add constraints
model.addConstr(x1 + x2 + x3 <= 3, "cons1")
model.addConstr(x1 + 4*x2 + 7*x3 <= 9, "cons2")
model.addConstr(x1 + x2 + x3 <= 1000, "dc")
model.addConstr(x1 >= 0, "cons3")
model.addConstr(x2 >= 0, "cons4")
model.addConstr(x3 >= 0, "cons5")

# Set objective function
model.setObjective(2*x1 + 3*x2 + x3, gp.GRB.MAXIMIZE)

# Solve the model
model.optimize()

# Access the sensitivity analysis
for var in model.getVars():
    print(f"Variable {var.varName} has optimal value {var.x}")
    print(f"  Coefficient sensitivity: [{var.SAObjLow}, {var.SAObjUp}]")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 3 columns and 12 nonzeros
Model fingerprint: 0x562ff4f8
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+03]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+00   1.121750e+00   0.000000e+00      0s
       2    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  8.000000000e+00
Variable x1 has optimal value 1.0
  Coefficient sensitivity: [0.7499999999999998, 3.0]
Variable x2 has optimal value 2.0

In [116]:
# Get shadow price
for constr in model.getConstrs():
    print(f"{constr.constrName}: {constr.Pi}")

# Get slack values of the constraints
print("\nSlack Values:")
for constr in model.getConstrs():
    print(f"{constr.constrName}: {constr.Slack}")
    
#Get range of rhs for constaraints to maintain optimality
for constr in model.getConstrs():
    print(f'Constraint {constr.ConstrName}:')
    print(f'  RHS lower bound: {constr.SARHSLow}')
    print(f'  RHS upper bound: {constr.SARHSUp}')

cons1: 1.6666666666666667
cons2: 0.3333333333333333
dc: 0.0
cons3: 0.0
cons4: 0.0
cons5: 0.0

Slack Values:
cons1: 0.0
cons2: 0.0
dc: 997.0
cons3: -1.0
cons4: -2.0
cons5: -0.0
Constraint cons1:
  RHS lower bound: 2.25
  RHS upper bound: 9.000000000000002
Constraint cons2:
  RHS lower bound: 3.0
  RHS upper bound: 12.0
Constraint dc:
  RHS lower bound: 3.0
  RHS upper bound: inf
Constraint cons3:
  RHS lower bound: -inf
  RHS upper bound: 1.0
Constraint cons4:
  RHS lower bound: -inf
  RHS upper bound: 2.0
Constraint cons5:
  RHS lower bound: -inf
  RHS upper bound: 0.0
